In [1]:
%%writefile kernel.cu

Writing kernel.cu


In [7]:
%%writefile kernel.cu
#include <iostream>
#include <cmath>
#include <cstdlib>
#include <ctime>
#include <cuda_runtime.h>
#include <cstdio> // Include for printf

#ifndef M_PI
#define M_PI 3.14159265358979323846
#endif

// Define number of clusters and data points
#define NUM_CLUSTERS 2
#define N 1024
#define THREADS_PER_BLOCK 256
#define CUDA_CHECK(call) \
    do { \
        cudaError_t error = call; \
        if (error != cudaSuccess) { \
            std::cerr << "CUDA Error: " << cudaGetErrorString(error) \
                      << " at " << __FILE__ << ":" << __LINE__ << std::endl; \
            exit(EXIT_FAILURE); \
        } \
    } while (0)
__global__ void eStepKernel(float* data, int N_data, float* mu, float* sigma,
                           float* pival, float* responsibilities) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx < N_data) {
        float x = data[idx];
        float probs[NUM_CLUSTERS];
        float sum = 0.0f;

        // printf("E-step: idx = %d, x = %f\n", idx, x); // Added printf

        for (int k = 0; k < NUM_CLUSTERS; k++) {
            // printf("E-step: Cluster %d, mu = %f, sigma = %f, pival = %f\n", k, mu[k], sigma[k], pival[k]); // Added printf
            float diff = x - mu[k];
            float exponent = -0.5f * (diff * diff) / (sigma[k] * sigma[k]);
            float gauss = (1.0f / (sqrtf(2.0f * M_PI) * sigma[k])) * expf(exponent);
            probs[k] = pival[k] * gauss;
            sum += probs[k];
            // printf("E-step: Cluster %d, diff = %f, exponent = %f, gauss = %f, prob = %f, sum = %f\n", k, diff, exponent, gauss, probs[k], sum); // Added printf
        }

        for (int k = 0; k < NUM_CLUSTERS; k++) {
            responsibilities[idx * NUM_CLUSTERS + k] = probs[k] / sum;
            // printf("E-step: idx = %d, Cluster %d, responsibility = %f\n", idx, k, responsibilities[idx * NUM_CLUSTERS + k]);
        }
    }
}
__global__ void mStepKernel(float* data, int N_data, float* responsibilities,
                           float* sum_gamma, float* sum_x, float* sum_x2) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx < N_data) {
        float x = data[idx];
        for (int k = 0; k < NUM_CLUSTERS; k++) {
            float gamma = responsibilities[idx * NUM_CLUSTERS + k];
            atomicAdd(&sum_gamma[k], gamma);
            atomicAdd(&sum_x[k], gamma * x);
            atomicAdd(&sum_x2[k], gamma * x * x);
            // printf("M-step: idx = %d, Cluster = %d, gamma = %f, sum_gamma = %f, sum_x = %f, sum_x2 = %f\n", idx, k, gamma, sum_gamma[k], sum_x[k], sum_x2[k]); // Added printf
        }
    }
}

int main() {
    // Seed the random number generator
    srand(static_cast<unsigned>(time(NULL)));

    float h_data[N];
    for (int i = 0; i < N; i++) {
        if (i < N/2) {
            h_data[i] = 2.0f + static_cast<float>(rand()) / RAND_MAX;
        } else {
            h_data[i] = 8.0f + static_cast<float>(rand()) / RAND_MAX;
        }
    }

    // Initial parameters (host)
    float h_mu[NUM_CLUSTERS] = {1.0f, 9.0f};
    float h_sigma[NUM_CLUSTERS] = {1.0f, 1.0f};
    float h_pival[NUM_CLUSTERS] = {0.5f, 0.5f};

    float *d_data, *d_mu, *d_sigma, *d_pival;
    float *d_responsibilities, *d_sum_gamma, *d_sum_x, *d_sum_x2;

    CUDA_CHECK(cudaMalloc(&d_data, N * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_mu, NUM_CLUSTERS * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_sigma, NUM_CLUSTERS * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_pival, NUM_CLUSTERS * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_responsibilities, N * NUM_CLUSTERS * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_sum_gamma, NUM_CLUSTERS * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_sum_x, NUM_CLUSTERS * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_sum_x2, NUM_CLUSTERS * sizeof(float)));

    CUDA_CHECK(cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_mu, h_mu, NUM_CLUSTERS * sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_sigma, h_sigma, NUM_CLUSTERS * sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_pival, h_pival, NUM_CLUSTERS * sizeof(float), cudaMemcpyHostToDevice));

    int blocks = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

    float h_sum_gamma[NUM_CLUSTERS];
    float h_sum_x[NUM_CLUSTERS];
    float h_sum_x2[NUM_CLUSTERS];

    // EM iterations
    int maxIter = 100;
    for (int iter = 0; iter < maxIter; iter++) {

        eStepKernel<<<blocks, THREADS_PER_BLOCK>>>(d_data, N, d_mu, d_sigma,
                                                  d_pival, d_responsibilities);
        CUDA_CHECK(cudaDeviceSynchronize());

        CUDA_CHECK(cudaMemset(d_sum_gamma, 0, NUM_CLUSTERS * sizeof(float)));
        CUDA_CHECK(cudaMemset(d_sum_x, 0, NUM_CLUSTERS * sizeof(float)));
        CUDA_CHECK(cudaMemset(d_sum_x2, 0, NUM_CLUSTERS * sizeof(float)));

        mStepKernel<<<blocks, THREADS_PER_BLOCK>>>(d_data, N, d_responsibilities,
                                                  d_sum_gamma, d_sum_x, d_sum_x2);
        CUDA_CHECK(cudaDeviceSynchronize());

        CUDA_CHECK(cudaMemcpy(h_sum_gamma, d_sum_gamma, NUM_CLUSTERS * sizeof(float), cudaMemcpyDeviceToHost));
        CUDA_CHECK(cudaMemcpy(h_sum_x, d_sum_x, NUM_CLUSTERS * sizeof(float), cudaMemcpyDeviceToHost));
        CUDA_CHECK(cudaMemcpy(h_sum_x2, d_sum_x2, NUM_CLUSTERS * sizeof(float), cudaMemcpyDeviceToHost));

        for (int k = 0; k < NUM_CLUSTERS; k++) {
            if (h_sum_gamma[k] > 1e-6f) {
                // Update mean
                h_mu[k] = h_sum_x[k] / h_sum_gamma[k];


                float variance = h_sum_x2[k] / h_sum_gamma[k] - h_mu[k] * h_mu[k];
                h_sigma[k] = sqrtf(fmax(variance, 1e-6f));

                h_pival[k] = h_sum_gamma[k] / N;
            }
        }

        CUDA_CHECK(cudaMemcpy(d_mu, h_mu, NUM_CLUSTERS * sizeof(float), cudaMemcpyHostToDevice));
        CUDA_CHECK(cudaMemcpy(d_sigma, h_sigma, NUM_CLUSTERS * sizeof(float), cudaMemcpyHostToDevice));
        CUDA_CHECK(cudaMemcpy(d_pival, h_pival, NUM_CLUSTERS * sizeof(float), cudaMemcpyHostToDevice));

        if (iter % 10 == 0) {
            std::cout << "Iteration " << iter << ":\n";
            for (int k = 0; k < NUM_CLUSTERS; k++) {
                std::cout << "Cluster " << k << ": "
                         << "mu = " << h_mu[k] << ", "
                         << "sigma = " << h_sigma[k] << ", "
                         << "pi = " << h_pival[k] << std::endl;
            }
            std::cout << std::endl;
        }
    }

    cudaFree(d_data);
    cudaFree(d_mu);
    cudaFree(d_sigma);
    cudaFree(d_pival);
    cudaFree(d_responsibilities);
    cudaFree(d_sum_gamma);
    cudaFree(d_sum_x);
    cudaFree(d_sum_x2);

    return 0;
}

Overwriting kernel.cu


In [8]:
!nvcc kernel.cu -o kernel -gencode arch=compute_75,code=sm_75 -lcublas

!/content/kernel

Iteration 0:
Cluster 0: mu = 2.51149, sigma = 0.296621, pi = 0.5
Cluster 1: mu = 8.49133, sigma = 0.29203, pi = 0.5

Iteration 10:
Cluster 0: mu = 2.51149, sigma = 0.296624, pi = 0.5
Cluster 1: mu = 8.49133, sigma = 0.292069, pi = 0.5

Iteration 20:
Cluster 0: mu = 2.51149, sigma = 0.29662, pi = 0.5
Cluster 1: mu = 8.49133, sigma = 0.292069, pi = 0.5

Iteration 30:
Cluster 0: mu = 2.51149, sigma = 0.29662, pi = 0.5
Cluster 1: mu = 8.49133, sigma = 0.292056, pi = 0.5

Iteration 40:
Cluster 0: mu = 2.51149, sigma = 0.29662, pi = 0.5
Cluster 1: mu = 8.49133, sigma = 0.292056, pi = 0.5

Iteration 50:
Cluster 0: mu = 2.51149, sigma = 0.29662, pi = 0.5
Cluster 1: mu = 8.49133, sigma = 0.292056, pi = 0.5

Iteration 60:
Cluster 0: mu = 2.51149, sigma = 0.29662, pi = 0.5
Cluster 1: mu = 8.49133, sigma = 0.292056, pi = 0.5

Iteration 70:
Cluster 0: mu = 2.51149, sigma = 0.296624, pi = 0.5
Cluster 1: mu = 8.49133, sigma = 0.292069, pi = 0.5

Iteration 80:
Cluster 0: mu = 2.51149, sigma = 0.29662,